In [1]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
from transformers import AutoTokenizer, TextStreamer
model_name_or_path = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM
import torch

model_name_or_path = "gpt2"
model_base = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,    
)

In [82]:
prompt = "Start counting from 0: One, two"

inputs = tokenizer(prompt, return_tensors="pt").to(model_base.device)

streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True, 
    skip_special_tokens=True
)

outputs = model_base.generate(
    **inputs, 
    streamer=streamer, 
    use_cache=True, 
    max_new_tokens=64,     
    do_sample=True,
    temperature=1.0,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


, three, four, five, six, seven, eight, nine, ten, ten again. This means that you can count three, five and six, and nine times out of ten, from 0: One to Nine numbers if you like. Then you can also count from zero by using (3<<3)


In [137]:
import os
from pathlib import Path
import boto3

def download_s3_checkpoint(s3_bucket: str, s3_path: str, local_root: str = "./checkpoints") -> str:
    """
    Download a whole checkpoint folder from S3 using a simple path string.

    Args:
        s3_bucket: S3 bucket name, e.g. "neptune-examples"
        s3_path:   Path like "transformers/run1/6000" or "transformers/run1/checkpoint-6000"
                   (function normalizes to checkpoint-<N>)
        local_root: Local root where the checkpoint dir will be placed (default: ./outputs)

    Returns:
        The local checkpoint directory path.

    Raises:
        FileNotFoundError if the checkpoint folder does not exist.
        ValueError for malformed inputs.
    """
    # Normalize path pieces
    s3_path = s3_path.strip().strip("/")
    parts = s3_path.split("/")
    if len(parts) < 2:
        raise ValueError("s3_path must look like 'transformers/<run_id>/<checkpoint>'")

    base_prefix = "/".join(parts[:-1])                     # e.g. transformers/run1
    ckpt_part   = parts[-1]                                # e.g. 6000 or checkpoint-6000
    if ckpt_part.startswith("checkpoint-"):
        ckpt_name = ckpt_part
    else:
        if not ckpt_part.isdigit():
            raise ValueError("checkpoint must be an integer or 'checkpoint-<N>'")
        ckpt_name = f"checkpoint-{int(ckpt_part)}"

    prefix = f"{base_prefix}/{ckpt_name}/"                 # final S3 prefix to download

    s3 = boto3.client("s3")

    # Existence check
    resp = s3.list_objects_v2(Bucket=s3_bucket, Prefix=prefix, MaxKeys=1)
    if resp.get("KeyCount", 0) == 0:
        raise FileNotFoundError(f"No objects found under s3://{s3_bucket}/{prefix}")

    # Local destination: ./outputs/<run_id>/checkpoint-XXXX/
    run_id = parts[-2]
    local_ckpt_dir = Path(local_root) / run_id / ckpt_name

    if not os.path.exists(local_ckpt_dir):
        local_ckpt_dir.mkdir(parents=True, exist_ok=True)
        # Download everything under the prefix
        paginator = s3.get_paginator("list_objects_v2")
        for page in paginator.paginate(Bucket=s3_bucket, Prefix=prefix):
            for obj in page.get("Contents", []):
                key = obj["Key"]
                rel = key[len(prefix):]
                if not rel:   # skip folder marker keys
                    continue
                dest = local_ckpt_dir / rel
                dest.parent.mkdir(parents=True, exist_ok=True)
                s3.download_file(s3_bucket, key, str(dest))
        print(f"⬇️ Downloaded to: {local_ckpt_dir}")
    else:
        print(f"✅ Using cached file at {local_ckpt_dir}")

    return str(local_ckpt_dir)

S3_BUCKET = "neptune-examples"
CHECKPOINT_STEP = "500"
RUN_ID = "detailed-observer-20250903100306767-ou2cg"

checkpoint_path   = f"models/{RUN_ID}/checkpoint-{CHECKPOINT_STEP}"   # or "transformers/run1/checkpoint-6000"
local_dir = download_s3_checkpoint(S3_BUCKET, checkpoint_path)


OSError: [Errno 28] No space left on device

In [126]:
# Load downloaded checkpoint
from transformers import AutoTokenizer, TextStreamer
model_name_or_path = local_dir
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM
import torch

model_name_or_path = local_dir
model2 = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,    
)

In [136]:
# Vibe check with downloaded checkpoint
prompt = "Start counting from 10: Eleven,"

inputs = tokenizer(prompt, return_tensors="pt").to(model2.device)

streamer = TextStreamer(
    tokenizer, 
    skip_prompt=False, 
    skip_special_tokens=True
)

outputs = model2.generate(
    **inputs, 
    streamer=streamer, 
    use_cache=True, 
    max_new_tokens=32,     
    do_sample=True,
    temperature=0.7,
    return_dict_in_generate=True,
)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Start counting from 10: Eleven, Twelve, Thirteen, ... to get to the last number in the sequence is 10!

• ## Math Algebra Problems

How many different solutions


In [89]:
# Get the full generated text (including prompt)
full_generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

# Get only the new tokens (excluding the original prompt)
prompt_length = inputs.input_ids.shape[1]
new_tokens = outputs.sequences[0][prompt_length:]
generated_response = tokenizer.decode(new_tokens, skip_special_tokens=True)
print(generated_response)

, three… if you count them all and find that there are all three, you have a very few more left.
 You can calculate the number of permutations that you would need to construct from the permutations by adding up the permutations you know from the number of permutations from the numbers of permutations.
